# ALA2005 Pipeline Design Guideline Augmented with a ReAct Agent

## Setup

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
# setup Arize Phoenix for logging/observability
import os
import llama_index.core

PHOENIX_API_KEY = os.getenv("PHOENIX_API_KEY")
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI as llma_OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-large")
llm = llma_OpenAI(model="gpt-4o")

Settings.embed_model = embed_model
Settings.llm = llm

In [4]:
from openai import OpenAI

client_openai = OpenAI()
openai_model = 'gpt-4o-mini'

## Parsing

In [5]:
from llama_parse import LlamaParse

parsingInstructionManga = """The provided document is a Guidelines for the Design of Buried Steel Pipe .
It does contain images, tables, math equations.
Output any math equation in LATEX markdown (between $$)."""

In [6]:
# print(f"Parsing PDF file...")
# parser_gpt4o = LlamaParse(result_type="markdown", gpt4o_mode=True, parsing_instruction=parsingInstructionManga, show_progress=True)
# md_json_objs = parser_gpt4o.load_data('docs\ALA2005-compressed_images.pdf')

In [7]:
# # Parse the documents using MarkdownElementNodeParser
# from llama_index.core.node_parser import MarkdownElementNodeParser

# node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)
# nodes = node_parser.get_nodes_from_documents(md_json_objs)

## Indexing

In [8]:
import os
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

In [9]:
if not os.path.exists("storage_nodes"):
    index = VectorStoreIndex(nodes, embed_model=embed_model)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage_nodes")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_nodes")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

In [10]:
QA_PROMPT_TMPL = """\

The document is the guideline to design buried pipelines which is parsed and converted into the 'markdown' mode.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query. Explain your reasoning for the final answer.
Output any math equation in LATEX markdown (between $$).
Always try to return the section number(s) where the information has been retrieved.

Query: {query_str}
Answer: """

# QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

In [18]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.base.response.schema import Response
from llama_index.core.prompts import PromptTemplate
from llama_index.core.schema import  MetadataMode

class MARKDOWN_QUERY_ENGINE(CustomQueryEngine):

    qa_prompt: PromptTemplate

    def __init__(self, qa_prompt = PromptTemplate(QA_PROMPT_TMPL), **kwargs) -> None:
        super().__init__(qa_prompt=qa_prompt, **kwargs)

    def custom_query(self, query_str: str):
        retriever = index.as_retriever(similarity_top_k=5)
        # retrieve text nodes
        text_nodes = retriever.retrieve(query_str)

        context_str = "\n\n".join(
                    [r.get_content(metadata_mode=MetadataMode.LLM) for r in text_nodes]
                )
        fmt_prompt = self.qa_prompt.format(context_str=context_str, query_str=query_str)
        llm_response = llm.complete(prompt=fmt_prompt)
        return Response(response=str(llm_response), source_nodes=text_nodes)
    

In [19]:
query_str = 'Find the surface live load transferred to the pipe from different height cover?'

query_engine = MARKDOWN_QUERY_ENGINE()

## ReAct Agent

In [20]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import ReActAgent

In [21]:
import importlib
import src.prompt_temp as prompt_temp
importlib.reload(prompt_temp)
from src.prompt_temp import react_system_prmopt

In [22]:
pipe_design_guideline_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="pipe_design_guideline",
    description=(
        "This is a guideline for the design of brined steel pipelines."
    ),
)
agent = ReActAgent.from_tools(
    [pipe_design_guideline_tool], llm=llm, verbose=True
)

# Load a custom system prompt
react_system_prompt = react_system_prmopt()

# Update the agent with the custom system prompt
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [23]:
# response = agent.query("Tell me about the different regions and subregions where Conoco Phillips has a production base.")
response = agent.chat(
    query_str
)

> Running step ec0f8c7c-d123-4850-8192-8d4e6ff174c1. Step input: Find the surface live load transferred to the pipe from different height cover?
Thought: I need to use the pipe design guideline to find the surface live load transferred to the pipe from different height covers.
Action: pipe_design_guideline
Action Input: {'input': 'surface live load transferred to the pipe from different height cover'}
Observation: The surface live load transferred to the pipe from different heights of cover is detailed in the table titled "Live Load Transferred to Pipe Based on Height of Cover." This table provides the live load transferred to a pipe (in pounds per square inch) for various heights of cover (in feet) under different loading conditions: Highway H20, Railway E80, and Airport.

Here is the relevant table:

| Height of cover, ft | Live load transferred to pipe, lb/in² | Height of cover, ft | Live load transferred to pipe, lb/in² |
|---------------------|-------------------------------------

In [24]:
from IPython.display import display, Markdown
display(Markdown(response.response))

Answer: The surface live load transferred to the pipe from different heights of cover is detailed in the table titled "Live Load Transferred to Pipe Based on Height of Cover." Here is a summary of the live load transferred to a pipe (in pounds per square inch) for various heights of cover (in feet) under different loading conditions: Highway H20, Railway E80, and Airport.

| Height of cover, ft | Highway H20 (lb/in²) | Railway E80 (lb/in²) | Airport (lb/in²) |
|---------------------|----------------------|----------------------|------------------|
| 1                   | 12.50                | --                   | --               |
| 2                   | 5.56                 | 26.39                | 13.14            |
| 3                   | 4.17                 | 23.61                | 12.28            |
| 4                   | 2.78                 | 18.40                | 11.27            |
| 5                   | 1.74                 | 16.67                | 10.09            |
| 6                   | 1.39                 | 15.63                | 8.79             |
| 7                   | 1.22                 | 12.15                | 7.85             |
| 8                   | 0.69                 | 11.11                | 6.93             |
| 10                  | --                   | 7.64                 | 6.09             |
| 12                  | --                   | 5.56                 | 4.76             |
| 14                  | --                   | 4.17                 | 3.06             |
| 16                  | --                   | 3.47                 | 2.29             |
| 18                  | --                   | 2.78                 | 1.91             |
| 20                  | --                   | 2.08                 | 1.53             |
| 22                  | --                   | 1.91                 | 1.14             |
| 24                  | --                   | 1.74                 | 1.05             |
| 26                  | --                   | 1.39                 | --               |
| 28                  | --                   | 1.04                 | --               |
| 30                  | --                   | 0.69                 | --               |
| 35                  | --                   | --                   | --               |
| 40                  | --                   | --                   | --               |

**Notes:**
- The live load becomes negligible for HS-20 loads when the earth cover exceeds 8 feet.
- The live load becomes negligible for E-80 loads when the earth cover exceeds 30 feet.
- The live load becomes negligible for airport loads when the earth cover exceeds 24 feet.

**Section Reference:**
- Section 4.0 Surface Live Loads
- Section 4.1 Applied Loads